In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

# Transferència de coneixement

L'objectiu d'avui és aprendre com podem emprar arquitectures ja existents per resoldre els nostres problemes. 

Com objectius secundaris tenim:

1. Conèixer un nou conjunt de dades
2. Entendre en profunditat com són dos dels models més emprats.
3. Guardar i carregar xarxes neuronals

## Dades

El conjunt de dades [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) consta de 60.000 imatges en color de 32x32 pixels etiquetades en 10 classes, amb 6.000 imatges per classe. Hi ha 50.000 imatges d'entrenament i 10.000 imatges de _test_.


Si voleu normalitzar les dades, a continuació teniu els valors ja calculats:

  - mitjana: (0.4914, 0.4822, 0.4465)
  - desviació típica: (0.247, 0.243, 0.261)

Una altra funció que pot ser útil és `Resize(mida_desti)` que rep un enter com a paràmetre (la mida final).


In [ ]:
train_batch_size = 64
test_batch_size = 100

# Definim una seqüència (composició) de transformacions 
transform=transforms.Compose([
    transforms.ToTensor() #,
    # podeu afegir aquí les vostres
    ])

# Descarregam un dataset ja integrat en la llibreria Pytorch
train_data = datasets.CIFAR10('../data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10('../data', train=False, transform=transform)

# Transformam les dades en l'estructura necessaria per entrenar una xarxa
train_loader = torch.utils.data.DataLoader(train_data, train_batch_size)
test_loader = torch.utils.data.DataLoader(test_data, test_batch_size)

## Transfer learning (Definició de la xarxa)

En aquesta pràctica aplicarem la tècnica de _transfer learning_ a partir de dues de les xarxes més conegudes en el camp de visió per computador:

-[**AlexNet**](https://proceedings.neurips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf). (ImageNet Classification with Deep Convolutional Neural Network, 2012). La mida d'entrada de les imatges és de (227x227x3).Té prop de 60 milions de paràmetres entrenables.

[**MobileNet v3**](https://pytorch.org/vision/main/models/mobilenetv3.html): és una arquitectura de xarxa neuronal convolucional dissenyada per a aplicacions de visió per ordinador en dispositius mòbils i sistemes encastats. Va ser desenvolupada per Google a la seva sèrie MobileNet, que se centra en l'eficiència i el rendiment de les xarxes neuronals en dispositius amb recursos limitats, com ara telèfons mòbils. Té dues versions: _small_ amb configuracions que van dels 2 als 5 milions de paràmetres; _large_ amb uns 6 milions de paràmetres. La mida d'entrada de les imatges és de (224x224x3)

_Pytorch_ ens permet emprar aquestes xarxes de manera molt senzilla. [Més informació](https://pytorch.org/vision/stable/models.html).

Si el model que cercam no es troba integrat dins la llibreria _Pytorch_ és bastant probable que si la trobem a Huggingface.

Descarregarem AlexNet i a analitzar-la. En aquest cas no només ens baixam la seva arquitectura, també els pesos resultants de l'entrenament.

In [ ]:
alex = models.alexnet(weights=True)

print("-"*50)
print("Arquitectura AlexNet")
print("-"*50)
print(alex)

Hi ha diverses maneres de realitzar la tècnica de _TransferLearning_ les dues més conegudes són:

 - **"Congelar"** els pesos de la part d'extracció de característiques (la part convolucional) i crear un nou classificador que s'adapti al nostre problema. Això implica que només entrenam una part de la xarxa.
 - **Reentrenar tota la xarxa**.

Com que la nostra capacitat de càlcul és limitada, ens decantarem per la primera opció. Pensau que

 Per tal d'evitar el reentrenament necessitam canviar el valor de l'atribut  `requires_grad` al valor `False`. Aquest atribut és propietat de cada tensor. Podem recorrer els tensors mitjançant el següent codi:
 ```
for param in alex.features.parameters():
    param.requires_grad = False
 ```
 ### Feina a fer:

- Bloc 1
 1. Carregar la xarxa AlexNet i seleccionar la part d'extracció de característiques.
 2. Definir un entorn seqüencial on implementarem el classificador de la xarxa.
 3. Realitzar un entrenament: comparar rendiment (accuracy) i nombre de paràmetres.
 4. Provar de guardar la vostra xarxa i tornar-la a carregar. Classificar una imatge del conjunt de test.
- Bloc 2: Repetir el mateix procés que l'anterior amb la xarxa mobilenet v3.


In [ ]:
#TODO Congelar

my_net =  nn.Sequential(#TODO posar ,
                        nn.Flatten(1,-1),
                        #TODO
                        )

## Entrenament

[shhht](https://github.com/tqdm/tqdm) si voleu canviar el resum de l'entrenament per una barra de progrés

In [ ]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100, verbose=True):
    
    model.train()

    loss_v = 0

    for batch_idx, (data, target) in enumerate(train_loader):
    
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target, reduction='mean') 
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Average: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), loss.item()/ len(data)))
        loss_v += loss.item()

    loss_v /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}\n'.format(loss_v))
 
    return loss_v


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='mean') 
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max probability
            correct += pred.eq(target.view_as(pred)).sum().item()
 
  
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return test_loss

In [ ]:
use_cuda = False
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

epochs = #¿?
lr = #¿?

model = my_net.to(device)

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) # !!!

print("Parameters ", pytorch_total_params)
optimizer = #¿?

# Guardam el valor de pèrdua mig de cada iteració (època)
train_l = np.zeros((epochs))
test_l = np.zeros((epochs))

# Bucle d'entrenament
for epoch in range(0, epochs):
    train_l[epoch] = train(model, device, train_loader, optimizer, epoch)
    test_l[epoch]  = test(model, device, test_loader)


In [ ]:
plt.title("Resultats de l'entrenament")
plt.plot(range(1, (epochs + 1)), train_l,  c="red", label="train")
plt.plot(range(1,  (epochs + 1)), test_l,  c="green", label="test")
plt.legend();